DL model to classify pokemons by generations

Dataset source: https://www.kaggle.com/datasets/vishalsubbiah/pokemon-images-and-types 

In [3]:
import os
import shutil
import pandas as pd
import numpy as np

import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [9]:
#load csv and add filenames
df = pd.read_csv("pokemon.csv")
df["Filename"] = df["Name"] + ".png"
df

,Name,Type1,Type2,Filename
0,bulbasaur,Grass,Poison,bulbasaur.png
1,ivysaur,Grass,Poison,ivysaur.png
2,venusaur,Grass,Poison,venusaur.png
3,charmander,Fire,NaN,charmander.png
4,charmeleon,Fire,NaN,charmeleon.png
...,...,...,...,...
804,stakataka,Rock,Steel,stakataka.png
805,blacephalon,Fire,Ghost,blacephalon.png
806,zeraora,Electric,NaN,zeraora.png
807,meltan,Steel,NaN,meltan.png


In [10]:
#Get valid types (any that has more than 50 occurrences)
# df.groupby("Type1")["Type1"].count().sort_values(ascending=False)
valid_types = list(df.groupby("Type1")["Type1"].count().loc[lambda x: x > 50].index)
print(f"Valid types: {valid_types}")

Valid types: ['Bug', 'Fire', 'Grass', 'Normal', 'Psychic', 'Water']


In [11]:
#Fitler pokemon by valid types
valid_pokemon = df[df["Type1"].isin(valid_types)];
valid_pokemon = valid_pokemon[["Name", "Type1", "Filename"]]
valid_pokemon.reset_index(inplace=True)

In [12]:
#Data split
train, valid, test = np.split(valid_pokemon, [int(0.6*len(valid_pokemon)), int(0.8*len(valid_pokemon))])    

In [18]:
#Create directories
original_dataset = "images/images/"
base_dir = "pokemon/"
os.makedirs(base_dir, exist_ok=True)

train_dir = os.path.join(base_dir, "train")
os.makedirs(train_dir, exist_ok=True)
validation_dir = os.path.join(base_dir, "validation")
os.makedirs(validation_dir, exist_ok=True)
test_dir = os.path.join(base_dir, "test")
os.makedirs(test_dir, exist_ok=True)

for type in valid_types:
	new_dir = os.path.join(train_dir, type)
	os.makedirs(new_dir, exist_ok=True)

for type in valid_types:
	new_dir = os.path.join(validation_dir, type)
	os.makedirs(new_dir, exist_ok=True)

for type in valid_types:
	new_dir = os.path.join(test_dir, type)
	os.makedirs(new_dir, exist_ok=True)

for index, row in train.iterrows():
	src = os.path.join(original_dataset, row["Filename"])
	folder = os.path.join(train_dir, row["Type1"])
	dst = os.path.join(folder, row["Filename"])
	shutil.copyfile(src, dst)

for index, row in train.iterrows():
	src = os.path.join(original_dataset, row["Filename"])
	folder = os.path.join(validation_dir, row["Type1"])
	dst = os.path.join(folder, row["Filename"])
	shutil.copyfile(src, dst)

for index, row in train.iterrows():
	src = os.path.join(original_dataset, row["Filename"])
	folder = os.path.join(test_dir, row["Type1"])
	dst = os.path.join(folder, row["Filename"])
	shutil.copyfile(src, dst)
